In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
import warnings

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

**Objective**
To leverage a transactional dataset and perform a comprehensive Exploratory Data Analysis (EDA) to gain deep insights into customer behaviour. This analysis will focus on uncovering customer preferences, identifying purchasing patterns, and understanding overall customer trends. These insights will be used to generate actionable recommendations for Supermarkets to improve business, ultimately enhancing profits and better customer retention.

**Scope**
The scope of the analysis is to gain insights from Nielsen store transaction data. The dataset includes information about monthly sales, store codes, bill IDs, bill amounts, quantities, values, prices, and categorical attributes such as group, subgroup, company name, mother brand, and brand. The analysis aims to understand sales trends, customer preferences, store performance, and the relationship between different variables.


In [ ]:
warnings.filterwarnings("ignore", "is_categorical_dtype")
warnings.filterwarnings("ignore", "use_inf_as_na")
warnings.filterwarnings("ignore", message="When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group.*")
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
#data_path = "/kaggle/input/nielsenhackathon/Hackathon_Working_Data.csv"
#data = pd.read_csv(data_path)

In [ ]:
data=pd.read_csv('/kaggle/input/store-transaction-data/Hackathon_Working_Data.csv')

In [ ]:
data.shape

In [ ]:
data.describe()

In [ ]:
data.info()

In [ ]:
data.tail(3)

In [ ]:
#To understand what is PRICE and Value
specific_value = 'T375'
filtered_data = data[data['BILL_ID'] == specific_value]
filtered_data=filtered_data[filtered_data['BILL_AMT']==420]
filtered_data

# 1. Understanding the data

* MONTH     : Month when the transaction occurred   
* STORECODE : A unique identifier for each store
* DAY       : Day when the transaction occurred  
* BILL_ID   : A unique identifier for each transaction or bill
* BILL_AMT  : Represents total amount of the purchase
* QTY       : Represents number of units of a product  
* VALUE     : The total value of the item(s) purchased, calculated as quantity multiplied by price 
* PRICE     : Price of a single unit  
* GRP       : Group or Category of the products  
* SGRP      : Subgroup or subcategory of product groups   
* SSGRP     : Subgroup of Subgroup of product groups 
* CMP       : Represents Company name
* MBRD      : Represents Mother Brand 
* BRD       : Represents Brand name

**Observation**
There are many same BILL_ID's that are generated for many stores
For example: BILL_ID : T375 is used in store 1,3,4,5 and 6. 
Hence, create a column which contains unique Bill_Id (helpful for sales per store analysis)

In [ ]:
len(data['GRP'].unique())

In [ ]:
data['UNIQUE_BILL_ID'] = data['STORECODE'] + data['BILL_ID']

In [ ]:
data.head(2)

In [ ]:
sales_revenue=data.groupby('UNIQUE_BILL_ID')['VALUE'].sum().reset_index()
sales_revenue.head()

In [ ]:
specific_value = 'N10T1048'
filtered_data = data[data['UNIQUE_BILL_ID'] == specific_value]
filtered_data

**Observation**: Use of Bill_AMT to compute sales revenue will produce false results because group by UNIQUE_BILL_ID or STORECODE will add the BILL_AMT if more than one item is purchased. Therefore, VALUE is the correct column to get actual revenue by the stores.

 # 2. Preparing Data for EDA

**Data preprocessing**
1. Missing Values
2. Duplicate Records
3. Data Encoding
4. Numerical Variables
5. Categorical Variables
6. Distribuiton of Numerical values
7. Univariate Analysis (Numerical & Categorical)
8. Correlation Analysis (Numerical & Categorical)


**1. Missing Values**

In [ ]:
#check null values
dataset= data.copy()
dataset.isnull().sum()

In [ ]:
#To check if there are any missing values
is_any_missing_data=dataset.isna().any().any()
is_any_missing_data

There are no missing values

**2. Duplicate Records**

In [ ]:
dataset.duplicated().sum()
#Fetch all duplicate values
#dr=dataset[dataset.duplicated(keep='last')]

In [ ]:
#Remove Duplicate Records
dataset.drop_duplicates(inplace=True)
dataset.duplicated().sum()

In [ ]:
dataset.dtypes

**3. Encoding data into numerical values**

There are 2 categorial columns (STORECODE,MONTH). Enocding these columns is a crucial step in preparing data for machine learning models, as most models require numerical input

*1. Enocde MONTH to numberical data*

In [ ]:
def remove_character(column):
    return column.str.replace('M', '')

dataset['MONTH'] = remove_character(dataset['MONTH'])
dataset['MONTH'] = dataset['MONTH'].astype(int)

*2. Another approach: Encode STORECODE using LabelEncoder*

In [ ]:
label_encoder = LabelEncoder()
dataset['STORECODE'] = label_encoder.fit_transform(dataset['STORECODE']) + 1

**4. Numerical features**

In [ ]:
numerical_variables = [feature for feature in dataset.columns if dataset[feature].dtypes!='O']
print('Number of numberical variables:', len(numerical_variables))

dataset[numerical_variables].sample(1)

In [ ]:
days=dataset['DAY'].unique()
print('days ',sorted(days))

In [ ]:
unique_storecodes = dataset['STORECODE'].unique()
sorted_unique_storecodes = sorted(unique_storecodes)
print('storecodes ',sorted_unique_storecodes)

**5. Categorical Variables**

In [ ]:
categorial_variables = [feature for feature in dataset.columns if dataset[feature].dtypes=='O']
print('Number of categorial variables:', len(categorial_variables))

dataset[categorial_variables].sample(1)

**6. Distribuition of numerical features**

**Distribution of continuous numerical features in your dataset**

In [ ]:
continuous_features = ['QTY', 'VALUE', 'PRICE']
for feature in continuous_features:
    sns.kdeplot(data[feature], shade=True, label=feature)
plt.title('KDE Plot of Continuous Features')
plt.xlabel('Value')
plt.ylabel('Density')
plt.xlim(0, 500)
plt.legend()
plt.show()

***a. Overall Sales by Month***

The month M2 has produced highest revenue

In [ ]:
Overall_sales_per_month= dataset.groupby('MONTH')['VALUE'].sum()
Overall_sales_per_month.plot(kind='line', marker='o', linestyle='-')
plt.xlabel('Month')
plt.ylabel('Overall Sales')
plt.title('Overall Sales per Month')
plt.show()

***b. Overall sales by store***

Store N8 has the highest revenue in all 3 months, whereas store N7 has the lowest revenue off all 10 stores. When it comes to avareage sales in each store, N3 has the highest average sales

In [ ]:
sales_per_store = dataset.groupby('STORECODE')['VALUE'].sum().reset_index()
sales_per_store.sort_values(by='VALUE',ascending=False,inplace=True)
sales_per_store.head(3)

In [ ]:
Average_sales_per_store = dataset.groupby('STORECODE')['VALUE'].mean().reset_index()
Average_sales_per_store.sort_values(by='VALUE',ascending=False,inplace=True)
Average_sales_per_store.head(3)

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(12, 10))
# Pie chart
explode = [0.1, 0,0,0,0,0,0,0,0,0]
axes[0, 0].pie(sales_per_store['VALUE'], labels=sales_per_store['STORECODE'],explode=explode, autopct='%1.1f%%')
axes[0, 0].set_title('Overall Sales by Store')
axes[0, 0].axis('equal')
# Bar plot
sns.barplot(x='STORECODE', y='VALUE', data=sales_per_store, palette="rocket", ax=axes[0, 1])
axes[0, 1].set_title('Overall Sales per Store')
axes[0, 1].set_xlabel('Store Code')
axes[0, 1].set_ylabel('Total Sales')
# Pie chart
axes[1, 0].pie(Average_sales_per_store['VALUE'], labels=Average_sales_per_store['STORECODE'],explode=explode, autopct='%1.1f%%')
axes[1, 0].set_title('Average Sales by Store')
axes[1, 0].axis('equal')
# Bar plot
sns.barplot(x='STORECODE', y='VALUE', data=Average_sales_per_store, palette="rocket", ax=axes[1, 1])
axes[1, 1].set_title('Average Sales per Store')
axes[1, 1].set_xlabel('Store Code')
axes[1, 1].set_ylabel('Total Sales')
plt.tight_layout()
plt.show()

***c. Daily sales by store***

In [ ]:
sales_by_month_day = dataset.groupby(['MONTH', 'DAY'])['VALUE'].sum().reset_index()

In [ ]:
fig, ax = plt.subplots(figsize=(12, 6))
palettes=['mediumorchid','pink','deepskyblue']
sns.lineplot(data=sales_by_month_day, x='DAY', y='VALUE', hue='MONTH', marker='o',markeredgecolor='black',palette=palettes, ax=ax)
ax.set_xlabel('Day')
ax.set_ylabel('Sales')
ax.set_title('Sales by Day, Month-wise')
ax.legend(title='Month')
ax.set_xticks(range(1, max(dataset['DAY']) + 1)) 

sales_by_month_day_m1 = sales_by_month_day[sales_by_month_day['MONTH'] == 1]
sales_by_month_day_m2 = sales_by_month_day[sales_by_month_day['MONTH'] == 2]
sales_by_month_day_m3 = sales_by_month_day[sales_by_month_day['MONTH'] == 3]

ax.fill_between(sales_by_month_day_m1['DAY'], sales_by_month_day_m1['VALUE'], alpha=0.5, label='M1',color="violet")
ax.fill_between(sales_by_month_day_m2['DAY'], sales_by_month_day_m2['VALUE'], alpha=0.5, label='M2',color="lightpink")
ax.fill_between(sales_by_month_day_m3['DAY'], sales_by_month_day_m3['VALUE'], alpha=0.5, label='M3',color="skyblue")
ax.set_xlabel('Day')
ax.set_ylabel('Total Sales')
ax.set_title('Total Sales by Month and Day')
ax.legend()
ax.grid(True,color='whitesmoke')
plt.tight_layout()
plt.show()

***d. Total number of products sold over 3 months***

In [ ]:
total_products_per_store= dataset.groupby('STORECODE')['QTY'].sum().sort_values(ascending=False).reset_index()

In [ ]:
sns.barplot( x='STORECODE', y='QTY',data=total_products_per_store, palette = "mako")
plt.title('Total Product sold per Store')
plt.xlabel('Store Code')
plt.ylabel('Total Products')
plt.grid(axis='y')
plt.show()

***e. Sales Vs Number of Products Sold on each store***

In [ ]:
merged_data = pd.merge(total_products_per_store, sales_per_store, on='STORECODE')
merged_data.set_index('STORECODE', inplace=True)
merged_data.reset_index(inplace=True)

In [ ]:
plt.figure(figsize=(12, 6))
bar_width = 0.35
x = range(len(merged_data))
bar1 = plt.bar(x, merged_data['QTY'], bar_width, label='Total Products')
bar2 = plt.bar(merged_data.index + bar_width, merged_data['VALUE'], bar_width, label='Total Sales', color='red')
plt.xlabel('Store')
plt.ylabel('Sales')
plt.title('Total Number of Products Sold and Total Sales per Store')
plt.xticks(x,merged_data['STORECODE'])
plt.grid(axis='y')
plt.legend()
plt.tight_layout()
plt.show()

***f. Total orders per day***

In [ ]:
orders_per_store_per_day_per_month = dataset.groupby(['STORECODE', 'MONTH', 'DAY'])['UNIQUE_BILL_ID'].nunique().reset_index(name='Total_Number_of_Orders')

In [ ]:
months = dataset['MONTH'].unique()

plt.figure(figsize=(12, 6 * len(months)))
palettes=['Greens','twilight','twilight_shifted','RdBu']
for i, month in enumerate(months, 1):
    month_data = dataset[dataset['MONTH'] == month]
    day_sales_month = month_data.groupby('DAY')['PRICE'].sum().reset_index()
    
    plt.subplot(len(months), 1, i)
    sns.barplot(x='DAY', y='PRICE', data=day_sales_month,palette=palettes[i])
    plt.xlabel('Day')
    plt.ylabel('Sales')
    plt.title(f"Month {month}: Day-wise Sales")

plt.tight_layout()
plt.show()

**g. The most bought product**

In [ ]:
most_bought_product = dataset['GRP'].value_counts().idxmax()
frequency = dataset['GRP'].value_counts().max()

print(f"The most bought product is '{most_bought_product}' with a frequency of {frequency}.")

In [ ]:
unique_grp_count = dataset['GRP'].nunique()
print("Unique GRP count:", unique_grp_count)

 **h. Top 10 Products that Customers most preferred in each store**

In [ ]:
stores = dataset['STORECODE'].unique()

plt.figure(figsize=(12, 5 * len(stores)))
palettes = ['hls', 'RdPu', 'Blues', 'Greens']

for i, store in enumerate(stores, 1):
    store_data = dataset[dataset['STORECODE'] == store]
    most_bought_product_in_each_store = store_data.groupby(['STORECODE', 'GRP'])['QTY'].sum().sort_values(ascending=False).reset_index()
    plt.subplot(len(stores), 1, i)
    sns.barplot(x='QTY', y='GRP', data=most_bought_product_in_each_store[:10], palette='plasma')
    plt.grid(axis='x')
    plt.xlabel('Units Sold')
    plt.ylabel('Product')
    plt.title(f"Store {store}: Top 10 Products Customers preferred")
plt.tight_layout()
plt.show()


**i.Top 3 products sold in each store**

In [ ]:
most_bought_product_in_each_store = dataset.groupby(['STORECODE', 'GRP'])['QTY'].sum().sort_values(ascending=False).reset_index()
most_bought_product_in_each_store.head()

In [ ]:
top_3_products = most_bought_product_in_each_store.groupby("STORECODE").head(3)
top_3_products=top_3_products.sort_values(by='STORECODE', ascending=True)

In [ ]:
prd_number=[]
prd_to_assign=['P1','P2','P3']
for i in range(0,int(len(top_3_products.index)/3)):
    prd_number.extend(prd_to_assign)
top_3 =top_3_products.copy()
top_3['PRD'] = prd_number
top_3.head()

In [ ]:
top_3_products_in_each_store=[]
top_3_products_in_each_store = top_3['GRP'].tolist()

In [ ]:
catplot = sns.catplot(x='STORECODE', y='QTY', hue='PRD', data=top_3, kind='bar', height=6, aspect=2,palette='YlGnBu')
ax1 = catplot.ax
annotations = top_3_products_in_each_store
for i, bar in enumerate(ax1.patches): 
    x_coord = bar.get_x() + bar.get_width() / 2 
    y_coord = bar.get_height() + 5 
    annotation_text = annotations[i]
    ax1.text(x_coord,y_coord, annotation_text,rotation=90,fontsize=9, ha='left', va='bottom')
ax1.set_xlabel('Storecode', fontsize=12)
ax1.set_ylabel('product Units sold', fontsize=12)
legend = catplot._legend
legend.set_title('PRD')
legend.get_title().set_fontsize(12)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.grid(axis='y')
plt.gcf().set_size_inches(10, 10)
plt.show()

In [ ]:
top_grp_N1 = top_3_products[top_3_products['STORECODE'] == 1]['GRP'].iloc[0]
print(f"Top GRP for store N1: {top_grp_N1}")

# Univariate Analysis

In [ ]:
for col in numerical_variables:
    print(col)
    print('Skew :', round(dataset[col].skew(), 2))
    plt.figure(figsize = (10, 4))
    plt.subplot(1, 2, 1)
    data[col].hist(grid=True,color='purple')
    plt.ylabel('count')
    plt.title('Histogram of ' + col)
    plt.subplot(1, 2, 2)
    sns.boxplot(x=dataset[col],color='deeppink')
    plt.title('Boxplot of ' + col)
    plt.show()

In [ ]:
dataset[categorial_variables].sample(1)

**Univariate Analysis on Categorical variables**
1. Most sold product GRP is Biscuts
2. Cream is most bought sub category of products
3. The most prefered products by the customers are from Hindustan Uniliver Limited
4. The most products are sold by TATA,DAIRY MILK and AMUL

In [ ]:
fig, axes = plt.subplots(3, 2, figsize = (18, 18),gridspec_kw={'hspace': 0.9})
fig.suptitle('Bar plot for all categorical variables in the dataset')
sns.countplot(ax = axes[0, 0], x = 'GRP', data = data, color = 'blue', 
              order = data['GRP'].head(20).value_counts().index);
sns.countplot(ax = axes[0, 1], x = 'SGRP', data = data, color = 'blue', 
              order = data['SGRP'].head(20).value_counts().index);
sns.countplot(ax = axes[1, 0], x = 'SSGRP', data = data, color = 'blue', 
              order = data['SSGRP'].head(20).value_counts().index);
sns.countplot(ax = axes[1, 1], x = 'CMP', data = data, color = 'blue', 
              order = data['CMP'].head(20).value_counts().index);
sns.countplot(ax = axes[2, 0], x = 'MBRD', data = data, color = 'blue', 
              order = data['MBRD'].head(20).value_counts().index);
sns.countplot(ax = axes[2, 1], x = 'BRD', data = data, color = 'blue', 
              order = data['BRD'].head(20).value_counts().index);
for ax in axes.flatten():
    ax.tick_params(labelrotation=90)
plt.show()


# Correlation Analysis

**Correlation between Numerical features**

In [ ]:
numerical_features = dataset.select_dtypes(include=['float64', 'int64'])
correlation_matrix = numerical_features.corr()
print("Correlation Matrix:")
print(correlation_matrix)

In [ ]:
plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f", linewidths=0.5)
plt.title('Correlation Matrix')
plt.show()

**Correlation between Categorical features**

In [ ]:
from scipy.stats import chi2_contingency
categorical_features = dataset.select_dtypes(include=['object'])
chi2_results = {}
for column1 in categorical_features:
    for column2 in categorical_features:
        if column1 != column2:
            contingency_table = pd.crosstab(dataset[column1], dataset[column2])
            chi2_stat, p_value, _, _ = chi2_contingency(contingency_table)
            chi2_results[(column1, column2)] = {'chi2_stat': chi2_stat, 'p_value': p_value}
for key, value in chi2_results.items():
   print(f"Chi-square test between {key}:")
   print(f"Chi-square statistic: {value['chi2_stat']}")
   print(f"P-value: {value['p_value']}")
   print()
